In [33]:
import keras
import pandas as pd
from keras.datasets import reuters


In [8]:
news = pd.read_json("News_Category_Dataset_v2.json", lines = True)
news.category = news.category.map(lambda x: "WORLDPOST" if x == "THE WORLDPOST" else x)
news['text'] = news['headline'] + " " + news['short_description']

In [9]:
X = news['text']
y =news['category']

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(y)
X = encoder.fit_transform(X)

seed = 40
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed, stratify=y)

In [28]:
print('# of Training Samples: {}'.format(len(x_train)))
print('# of Test Samples: {}'.format(len(x_test)))

num_classes = max(y_train) + 1
print('# of Classes: {}'.format(num_classes))

# of Training Samples: 160682
# of Test Samples: 40171
# of Classes: 40


In [75]:
#x_train

In [76]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(path='News_Category_Dataset_v2.json',num_words=None, test_split=0.2)
word_index = reuters.get_word_index(path="News_Category_Dataset_v2.json")

print('# of Training Samples: {}'.format(len(x_train)))
print('# of Test Samples: {}'.format(len(x_test)))
num_classes = max(y_train) + 1
print('# of Classes: {}'.format(num_classes))

A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 87aedbeb0cb229e378797a632c1997b6 so we will re-download the data.
2113536/2110848 [==============================] - 1s 0us/step
A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 4d44cc38712099c9e383dc6e5f11a921 so we will re-download the data.
557056/550378 [==============================] - 0s 1us/step
# of Training Samples: 8982
# of Test Samples: 2246
# of Classes: 46


In [92]:
index_to_word = {}
for key, value in word_index.items():
    index_to_word[value] = key
print(' '.join([index_to_word[x] for x in x_train[2]]))
print(y_train[1])

KeyError: 0.0

In [78]:
from keras.preprocessing.text import Tokenizer

max_words = 10000

tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(x_train[0])
print(len(x_train[0]))

print(y_train[0])
print(len(y_train[0]))

[0. 1. 0. ... 0. 0. 0.]
10000
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
46


In [63]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.metrics_names)

['loss', 'accuracy']


In [64]:
batch_size = 32
epochs = 3

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.1)
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 8083 samples, validate on 899 samples
Epoch 1/3
8083/8083 [==============================] - 13s 2ms/step - loss: 1.2926 - accuracy: 0.7202 - val_loss: 0.9617 - val_accuracy: 0.7964
Epoch 2/3
8083/8083 [==============================] - 12s 2ms/step - loss: 0.4937 - accuracy: 0.8885 - val_loss: 0.8481 - val_accuracy: 0.8109
Epoch 3/3
2246/2246 [==============================] - 0s 195us/step
Test loss: 0.855713007923544
Test accuracy: 0.8094390034675598


In [90]:
MAX_SEQUENCE_LENGTH = 10000
import numpy as np
new_complaint = ['One team won their throphy in football.']
seq = tokenizer.texts_to_sequences(new_complaint)
padded = keras.preprocessing.sequence.pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)

labels = news['category']
print(pred, labels[np.argmax(pred)])

[[0.02171437 0.02781551 0.02055546 0.04472966 0.03582662 0.01814958
  0.01922203 0.01928871 0.02478462 0.02151496 0.02142528 0.02547126
  0.01929421 0.02287822 0.0191706  0.01971425 0.02502525 0.01897368
  0.01967131 0.02948719 0.02751614 0.02019703 0.01910604 0.01954896
  0.02362109 0.01963722 0.01817041 0.02090266 0.02083889 0.01977346
  0.02204905 0.02131508 0.02228377 0.01786004 0.01958208 0.01706697
  0.02190941 0.02038641 0.02022657 0.01938035 0.01974881 0.02017283
  0.01866088 0.01835962 0.01787101 0.01910242]] ENTERTAINMENT


array([[0.02171437, 0.02781551, 0.02055546, 0.04472966, 0.03582662,
        0.01814958, 0.01922203, 0.01928871, 0.02478462, 0.02151496,
        0.02142528, 0.02547126, 0.01929421, 0.02287822, 0.0191706 ,
        0.01971425, 0.02502525, 0.01897368, 0.01967131, 0.02948719,
        0.02751614, 0.02019703, 0.01910604, 0.01954896, 0.02362109,
        0.01963722, 0.01817041, 0.02090266, 0.02083889, 0.01977346,
        0.02204905, 0.02131508, 0.02228377, 0.01786004, 0.01958208,
        0.01706697, 0.02190941, 0.02038641, 0.02022657, 0.01938035,
        0.01974881, 0.02017283, 0.01866088, 0.01835962, 0.01787101,
        0.01910242]], dtype=float32)

In [31]:
from keras.preprocessing.text import Tokenizer


max_words = 10000

tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='count')
x_test = tokenizer.sequences_to_matrix(int(x_test), mode='count')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(x_train[0])
print(len(x_train[0]))

print(y_train[0])
print(len(y_train[0]))

TypeError: 'numpy.int64' object is not iterable